# Gas

* Gas Reserves - TN CM 
* Gas Production - BN CM
* Gas Consumption - BN CM
* Gas Prices - USD/MN BTU
* Gas Import Exports - BN CM
* LNG Exports - BN CM
* LNG Imports - BN CM
* Pipeline Trade Movements - BN CM